In [71]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import TruncatedSVD

df = pd.read_csv('idmanual.csv')
#df=df.dropna()
df.head()

,Term ID,Class,Description,Status,Effective Date,Type,Notes,TM5,NCL Version
0,025-73,25,Cravats,A,4/2/1991,GOODS,05-14-2015: A cravat is a scarf of silk or fin...,NaN,"""07-1997"""
1,025-1226,25,Espadrilles,A,4/2/1991,GOODS,05-14-2015: An espadrille is a sandal usually ...,T,"""07-1997"""
2,016-2204,16,Printed short stories in the field of {indicat...,A,5/14/2015,GOODS,NaN,NaN,"""10-2015"""
3,025-1008,25,Cagoules,A,6/25/2009,GOODS,05-14-2015: A cagoule is a lightweight usually...,NaN,"""09-2007"""
4,025-180,25,Pinafores,A,4/2/1991,GOODS,05-14-2015: A pinafore is a sleeveless garment...,NaN,"""07-1997"""


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62357 entries, 0 to 62356
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Term ID         62357 non-null  object
 1   Class           62357 non-null  object
 2   Description     62357 non-null  object
 3   Status          62357 non-null  object
 4   Effective Date  62357 non-null  object
 5   Type            62357 non-null  object
 6   Notes           15408 non-null  object
 7   TM5             23044 non-null  object
 8   NCL Version     62357 non-null  object
dtypes: object(9)
memory usage: 4.3+ MB


In [73]:
df.duplicated(subset='Description').sum()

131

In [74]:
df = df.drop_duplicates(subset='Description')

In [75]:
df.isna()

,Term ID,Class,Description,Status,Effective Date,Type,Notes,TM5,NCL Version
0,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,True,False
3,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...
62352,False,False,False,False,False,False,True,True,False
62353,False,False,False,False,False,False,False,False,False
62354,False,False,False,False,False,False,True,True,False
62355,False,False,False,False,False,False,True,True,False


In [76]:
df.duplicated(subset='Description').sum()

0

In [77]:
sample_size = 15000
df = df.sample(n=sample_size, replace=False, random_state=490)

df = df.reset_index()
df = df.drop('index',axis=1)

In [78]:
df.head()

,Term ID,Class,Description,Status,Effective Date,Type,Notes,TM5,NCL Version
0,044-2770,44,Health care services for assisting individuals...,A,11/7/2019,SERVICES,NaN,T,"""11-2019"""
1,041-1606,41,Providing education courses relating to the tr...,M,8/25/2011,SERVICES,"On 8-25-2011, this 7-20-2004 entry was amended...",T,"""09-2007"""
2,009-2008,9,Decorative switch plate covers,A,10/1/2004,GOODS,NaN,NaN,"""08-2002"""
3,038-12,38,Audio broadcasting,A,4/2/1991,SERVICES,NaN,T,"""07-1997"""
4,043-4,43,Cocktail lounges,X,7/25/2013,SERVICES,This entry was transferred from Class 42 to Cl...,NaN,"""10-2013"""


In [79]:
df['Description'] = df['Description'].str.lower()
df['Type'] = df['Type'].str.lower()

In [80]:
df2 = df.drop(['Term ID','Class','Description','Status','Effective Date','Type'],axis=1)

df2['data'] = df2[df2.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(df2['data'].head())

0    T "11-2019"
1    T "09-2007"
2      "08-2002"
3    T "07-1997"
4      "10-2013"
Name: data, dtype: object


In [81]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df2['data'])

In [82]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [83]:
print(similarities)

[[1.  0.  0.  ... 0.  0.5 0. ]
 [0.  1.  0.  ... 1.  0.  0. ]
 [0.  0.  1.  ... 0.  0.  0. ]
 ...
 [0.  1.  0.  ... 1.  0.  0. ]
 [0.5 0.  0.  ... 0.  1.  0. ]
 [0.  0.  0.  ... 0.  0.  1. ]]


In [84]:
df = pd.DataFrame(similarities, columns=df['Description'], index=df['Description']).reset_index()

df.head()

Description,Description,health care services for assisting individuals to stop smoking,providing education courses relating to the travel industry,decorative switch plate covers,audio broadcasting,cocktail lounges,radar guns for sporting events,homeotherapeutic hcg (human chorionic gonadotropin) oral sprays for weight loss purposes,metal zip ties,sweetmeats,...,safety glass for building purposes,rental of construction machines and apparatus and providing information relating thereto,embroidery needles,advertising by transmission of on-line publicity for third parties through electronic communications networks,inner tubes for automobile tires,equipment maintenance claims payment service for business,statues of precious metals,solar-powered cooling units,anti-nauseants,"pencil sharpening machines, electric or non-electric"
0,health care services for assisting individuals...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,...,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0
1,providing education courses relating to the tr...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,decorative switch plate covers,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,audio broadcasting,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cocktail lounges,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5


In [85]:
input_description = 'cocktail lounges'
recommendations = pd.DataFrame(df.nlargest(11,input_description)['Description'])
recommendations = recommendations[recommendations['Description']!=input_description]
print(recommendations)


                                           Description
19   carbohydrates used as an ingredient in the man...
98                          emergency power generators
157                                        pickhammers
169  audio speakers that may be attached with adhes...
170  security services for individuals and business...
194  recycled plastics in extruded form for use in ...
201  testing to determine professional competency i...
237                              beryllium [glucinium]
270                         livestock farming services
292  synthetic polymer thin film coatings in the na...
